# Chapter 09: Signals

📈Join our community: https://discord.gg/wXjNPAc5BH

📚Read our book: https://www.amazon.com/gp/product/B09HG18CYL 

🖥️Quantreo's YouTube channel: https://www.youtube.com/channel/UCp7jckfiEglNf_Gj62VR0pw

In [4]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
import ta
warnings.filterwarnings("ignore")

In [5]:
mt5.initialize()

False

In [7]:
class MT5:

    def get_rates(symbol, n, timeframe=mt5.TIMEFRAME_D1):
        """ Function to import the data of the chosen symbol"""

        # Initialize the connection if there is not
        mt5.initialize()

        # Current date extract
        utc_from = datetime.now()

        # Import the data into a tuple
        rates = mt5.copy_rates_from(symbol, timeframe, utc_from, n)

        # Tuple to dataframe
        rates_frame = pd.DataFrame(rates)

        # Convert time in seconds into the datetime format
        rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')

        # Convert the column "time" in the right format
        rates_frame['time'] = pd.to_datetime(rates_frame['time'], format='%Y-%m-%d')

        # Set column time as the index of the dataframe
        rates_frame = rates_frame.set_index('time')
        return rates_frame

### Random signal

In [8]:
def random():
    values = [True, False]
    buy = np.random.choice(values)
    sell = not buy
    
    return buy, sell

In [9]:
random()

(True, False)

### SMA signal

In [10]:
def SMA_strategy(symbol):
    
    # Import / Features engineering
    df = MT5.get_rates(symbol, 70)
    
    # Create Resistance using a rolling max
    df["SMA fast"] = df["close"].rolling(30).mean()

    # Create Support using a rolling min
    df["SMA slow"] = df["close"].rolling(60).mean()

    # Signals
    condition_buy = df["SMA fast"].iloc[-1] > df["SMA slow"].iloc[-1]

    condition_sell = df["SMA fast"].iloc[-1] < df["SMA slow"].iloc[-1]
    
    buy = condition_buy
    sell = condition_sell
    

    return buy, sell

In [9]:
SMA_strategy("PETR4F")

(True, False)

In [13]:
symbol = 'PETR4F'

### Multiple conditions entry

In [14]:
def SMA_rsi_strategy(symbol):
    
    # Import / Features engineering
    df = MT5.get_rates(symbol, 70)
    
    # Create Resistance using a rolling max
    df["SMA fast"] = df["close"].rolling(30).mean()

    # Create Support using a rolling min
    df["SMA slow"] = df["close"].rolling(60).mean()

    # Create RSI
    df["rsi"] = ta.momentum.RSIIndicator(df["close"], window=10).rsi()
    
    df["rsi yersteday"] = df["rsi"].shift(1)

    # Signals
    condition_1_buy = df["SMA fast"].iloc[-1] > df["SMA slow"].iloc[-1]
    condition_2_buy = df["rsi"].iloc[-1] < df["rsi yersteday"].iloc[-1]

    condition_1_sell = df["SMA fast"].iloc[-1] < df["SMA slow"].iloc[-1]
    condition_2_sell = df["rsi"].iloc[-1] > df["rsi yersteday"].iloc[-1]
    
    buy = condition_1_buy and condition_2_buy
    sell = condition_1_sell and condition_2_sell
    

    return buy, sell

In [15]:
SMA_rsi_strategy("PETR4F")

KeyError: 'time'